# Classification 02

# Resampling and evaluation of prediction methods

#### Josep Fortiana 2019-10-29

The goal of this laboratory is to attain a practical knowledge of some of the available procedures to assess the quality of prediction methods, classification (categorical response) and regression (numerical response), with a particular emphasis on detecting and avoiding overfitting.

We consider four procedures or families of procedures: 
    
1. Hold-out: Split the dataset in two parts, a training subset and a 'test' subset. The first one is used to train (estimate or learn parameters of) the prediction method. Meanwhile, the 'test' subset is held out, keeping it apart to rule out the optimistic bias inherent in using the same samples for learning to predict and for evaluating goodness of prediction.

2. $k$-fold cross-validation,  

3. Leave-one-out (LOO), 

4. Bootstrap.

We already know the first procedure. The second one is a systematic repetition of hold-out in such a way that consecutively each individual is in a training subset and in a test subset. The third procedure is an extreme $k$-fold cross-validation, where $k=n$, the number of individuals in the sample, in which the test subset is a single individual and the training subset  the remaining $(n-1)$ restantes. 

The fourth procedure is an application of a much more general statistical concept, the _bootstrap,_ which we review in Section A, before going to its application with the other procedures.

# Datasets and prediction methods

# A. `wine` dataset

`wine` data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines.

They can be found in the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/), which hosts many documented data sets to be used as benchmarks in evaluating Machine Learning methods and algorithms. Alternatively, should the link be broken, you can find the `.csv` file in the Virtual Campus. The following description is taken from the UCI website:

I think that the initial data set had around 30 variables, but for some reason I only have the 13 dimensional version. I had a list of what the 30 or so variables were, but a.) I lost it, and b.), I would not know which 13 variables are included in the set.

The attributes are:

01. Alcohol

02. Malic acid

03. Ash

04. Alcalinity of ash

05. Magnesium

06. Total phenols

07. Flavonoids

08. Nonflavonoid phenols

09. Proanthocyanins

10. Color intensity

11. Hue

12. OD280/OD315 of diluted wines

13. Proline


In a classification context, this is a well posed problem with "well behaved" class structures. A good data set for first testing of a new classifier, but not very challenging.

Since the `.csv` file has no first row with variable names we must set `header=FALSE` in the `read.csv` call (see default values for the optional parameters in the help). 

The casting `as.factor()` command has the purpose of conveying the fact that this variable is qualitative, so the R interpreter can use it as such.


In [ ]:
wine.url<-"http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
#wine<-read.csv(wine.url,header=FALSE)
wine<-read.csv("wine.csv",header=FALSE)
colnames(wine)<-c("Type","Alcohol","Malic","Ash", "Alcalinity","Magnesium","Phenols","Flavonoids", 
                  "Nonflavonoids","Proanthocyanins","Color","Hue", "Dilution","Proline")
wine$Type <- as.factor(wine$Type)

## Aa. `wine` dataset and $k$-NN

### Aa1. Hold-out

Split the whole dataset into two subsets, `train` with $\approx60\%$ of data, and `test` the rest.

In [ ]:
n<-nrow(wine)
ntrain<-ceiling(0.6*n)
ntest<-n-ntrain
Itrain<-sample(1:n,ntrain,replace=FALSE)
wine.train<-wine[Itrain,]
wine.test<-wine[-Itrain,]
Xtrain<-as.matrix(wine.train[,-1])
ytrain<-wine.train[,1]
Xtest<-as.matrix(wine.test[,-1])
ytest<-wine.test[,1]

In [ ]:
#install.packages("class",dependencies=TRUE,repos="https://cloud.r-project.org")
require(class)

#### Confusion matrix and misclassification error estimate

In [ ]:
k<-5
y.hat<-knn(Xtrain,Xtest,ytrain,k )
C<-table("True"=ytest,"Predicted"=y.hat)
P.err<-(ntest-sum(diag(C)))/ntest
C
sprintf("For k =%d,   Prob.err = %5.2f ", k, round(P.err,2))

#### Which is the optimal $k$?

Redo the above computation with several values of $k$ and decide which is the optimal $k$ according to the probability of misclassification.

_Hint:_ After doing it _"by hand"_ you can try a systematic approach, building a table where the first column is `k` and the second column the estimated probability of misclassification.

### Aa.2. k-fold cross-validation

Here we perform $k$-fold cross-validation with some simple code. An alternative approach is to use functions from a dedicated R package. My personal prejudice is in favor of writing one's own code. Anyway you can see in the Appendix at the end of this notebook some possible such packages and you can judge by yourselves.

In [ ]:
kfoldIndexes<-function(n,k){
    l<-floor(n/k)
    Indexes<-c(1,(1:k)*l)
    Indexes[k+1]<-n
    return(Indexes)
    }

In [ ]:
# Decide a 'K' for K-fold cross-validation (uppercase to avoid notation clash with the k in k-NN)
n<-nrow(wine)
K<-5
J<-kfoldIndexes(n,K)
J
Lower<-J[-(K+1)]
Upper<-J[-1]
Lower
Upper
# Una random permutation of indexes
I<-sample(1:n)

In [ ]:
k<-7 # this is the 'k' in k-NN
# Repeat with several 'k' and choose an optimal value.
P.ERR<-rep(0,K)
for (fold in 1:K){
    Itest<-I[Lower[fold]:Upper[fold]]
    wine.test<-wine[Itest,]
    wine.train<-wine[-Itest,]
    Xtrain<-as.matrix(wine.train[,-1])
    ytrain<-wine.train[,1]
    Xtest<-as.matrix(wine.test[,-1])
    ytest<-wine.test[,1]
    y.hat<-knn(Xtrain,Xtest,ytrain,k)
    C<-table("True"=ytest,"Predicted"=y.hat)
    print(C)
    P.ERR[fold]<-(ntest-sum(diag(C)))/ntest
    }
round(P.ERR,3)
mean.p.err<-mean(P.ERR)
round(mean.p.err,3)

### Aa.3. Leave-one-out _(LOO)_

In [ ]:
k<-7
# Repeat with several 'k' and choose an optimal value.
g<-nlevels(wine$Type)
C<-matrix(0,nrow=g,ncol=g)
for (i in 1:n){
    wine.test<-wine[i,]
    wine.train<-wine[-i,]
    Xtrain<-as.matrix(wine.train[,-1])
    ytrain<-wine.train[,1]
    Xtest<-as.matrix(wine.test[,-1])
    ytest<-wine.test[,1]
    y.hat<-knn(Xtrain,Xtest,ytrain,k)
    C[ytest,y.hat]=C[ytest,y.hat]+1
    }
print(C)
p.err<-(n-sum(diag(C)))/n
round(p.err,3)

### Aa.4.  _bootstrap_

In [ ]:
k<-7
# Repeat with several 'k' and choose an optimal value.
n<-nrow(wine)
I<-1:n
# Number of bootstrap resamples
B<-10
P.ERR<-rep(0,B)
for (b in 1:B){
    Ib<-sample(I,n,replace = TRUE)
    oob<-I[is.na(match(I,Ib))]
    Itest<-oob
    ntest<-length(oob)
    print(ntest)
    wine.test<-wine[Itest,]
    wine.train<-wine[-Itest,]
    Xtrain<-as.matrix(wine.train[,-1])
    ytrain<-wine.train[,1]
    Xtest<-as.matrix(wine.test[,-1])
    ytest<-wine.test[,1]
    y.hat<-knn(Xtrain,Xtest,ytrain,k)
    C<-table("True"=ytest,"Predicted"=y.hat)
    print(C)
    P.ERR[b]<-(ntest-sum(diag(C)))/ntest
    }
round(P.ERR,3)
mean.p.err<-mean(P.ERR)
round(mean.p.err,3)

# B. `Auto` dataset

#### Description
Gas mileage, horsepower, and other information for 392 vehicles.

#### Format
A data frame with 392 observations on the following 9 variables.

01. `mpg`: miles per gallon.

02. `cylinders`: Number of cylinders between 4 and 8.

03. `displacement`: Engine displacement (cu. inches).

04. `horsepower`: Engine horsepower.

05. `weight`: Vehicle weight (lbs.).

06. `acceleration`: Time to accelerate from 0 to 60 mph (sec.).

07. `year`: Model year (modulo 100).

08. `origin`: Origin of car (1. American, 2. European, 3. Japanese).

09. `name`: Vehicle name.

The orginal data contained 408 observations but 16 observations with missing values were removed.

#### Source
This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University. The dataset was used in the 1983 American Statistical Association Exposition.

In [1]:
#install.packages("ISLR",dependencies=TRUE,repos="https://cloud.r-project.org")
require(ISLR)

Loading required package: ISLR


In [2]:
data(Auto)
str(Auto)

'data.frame':	392 obs. of  9 variables:
 $ mpg         : num  18 15 18 16 17 15 14 14 14 15 ...
 $ cylinders   : num  8 8 8 8 8 8 8 8 8 8 ...
 $ displacement: num  307 350 318 304 302 429 454 440 455 390 ...
 $ horsepower  : num  130 165 150 150 140 198 220 215 225 190 ...
 $ weight      : num  3504 3693 3436 3433 3449 ...
 $ acceleration: num  12 11.5 11 12 10.5 10 9 8.5 10 8.5 ...
 $ year        : num  70 70 70 70 70 70 70 70 70 70 ...
 $ origin      : num  1 1 1 1 1 1 1 1 1 1 ...
 $ name        : Factor w/ 304 levels "amc ambassador brougham",..: 49 36 231 14 161 141 54 223 241 2 ...


In [3]:
# Discard the 'name' variable, irrelevant for prediction
Auto<-Auto[,-9]
str(Auto)

'data.frame':	392 obs. of  8 variables:
 $ mpg         : num  18 15 18 16 17 15 14 14 14 15 ...
 $ cylinders   : num  8 8 8 8 8 8 8 8 8 8 ...
 $ displacement: num  307 350 318 304 302 429 454 440 455 390 ...
 $ horsepower  : num  130 165 150 150 140 198 220 215 225 190 ...
 $ weight      : num  3504 3693 3436 3433 3449 ...
 $ acceleration: num  12 11.5 11 12 10.5 10 9 8.5 10 8.5 ...
 $ year        : num  70 70 70 70 70 70 70 70 70 70 ...
 $ origin      : num  1 1 1 1 1 1 1 1 1 1 ...


In [4]:
table(Auto$origin)


  1   2   3 
245  68  79 

In [5]:
table(Auto$year)


70 71 72 73 74 75 76 77 78 79 80 81 82 
29 27 28 40 26 30 34 28 36 29 27 28 30 

In [6]:
table(Auto$cylinders)


  3   4   5   6   8 
  4 199   3  83 103 

### Ba.1 Hold-out

In [7]:
n<-nrow(Auto)
ntrain<-ceiling(0.6*n)
ntest<-n-ntrain
Itrain<-sample(1:n,ntrain,replace=FALSE)
Auto.train<-Auto[Itrain,]
Auto.test<-Auto[-Itrain,]

In [8]:
Auto.lm.01<-lm(mpg~.,data=Auto.train)

In [9]:
summary(Auto.lm.01)


Call:
lm(formula = mpg ~ ., data = Auto.train)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.2612 -2.1990 -0.1923  2.0492 12.8552 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -2.073e+01  6.266e+00  -3.308 0.001092 ** 
cylinders    -3.031e-01  4.353e-01  -0.696 0.486925    
displacement  1.213e-02  1.052e-02   1.152 0.250457    
horsepower   -2.788e-02  2.033e-02  -1.372 0.171459    
weight       -5.535e-03  8.634e-04  -6.411 8.23e-10 ***
acceleration  7.293e-02  1.320e-01   0.552 0.581282    
year          7.853e-01  6.885e-02  11.407  < 2e-16 ***
origin        1.334e+00  3.755e-01   3.554 0.000462 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.47 on 228 degrees of freedom
Multiple R-squared:  0.809,	Adjusted R-squared:  0.8031 
F-statistic: 137.9 on 7 and 228 DF,  p-value: < 2.2e-16


In [10]:
step(Auto.lm.01)

Start:  AIC=595.16
mpg ~ cylinders + displacement + horsepower + weight + acceleration + 
    year + origin

               Df Sum of Sq    RSS    AIC
- acceleration  1      3.67 2749.6 593.47
- cylinders     1      5.84 2751.8 593.66
- displacement  1     15.99 2761.9 594.53
- horsepower    1     22.67 2768.6 595.10
<none>                      2746.0 595.16
- origin        1    152.10 2898.0 605.88
- weight        1    494.98 3240.9 632.27
- year          1   1567.08 4313.0 699.71

Step:  AIC=593.47
mpg ~ cylinders + displacement + horsepower + weight + year + 
    origin

               Df Sum of Sq    RSS    AIC
- cylinders     1      6.63 2756.3 592.04
- displacement  1     15.71 2765.3 592.82
<none>                      2749.6 593.47
- horsepower    1     63.47 2813.1 596.86
- origin        1    154.39 2904.0 604.36
- weight        1    600.14 3349.8 638.06
- year          1   1564.63 4314.3 697.78

Step:  AIC=592.04
mpg ~ displacement + horsepower + weight + year + origin

      


Call:
lm(formula = mpg ~ horsepower + weight + year + origin, data = Auto.train)

Coefficients:
(Intercept)   horsepower       weight         year       origin  
 -19.756725    -0.026577    -0.004927     0.773919     1.194028  


In [11]:
mpg.hat<-predict(Auto.lm.01,newdata=Auto.test[,-1],type="response")

In [12]:
mpg.true<-Auto.test[,1]

In [13]:
HO.CV.RMS<-mean((mpg.true-mpg.hat)^2)
round(HO.CV.RMS,2)

[1] 10.08

In [14]:
Auto.lm.02<-lm(mpg~cylinders + displacement + horsepower + weight + year + origin,data=Auto.train)
summary(Auto.lm.02)


Call:
lm(formula = mpg ~ cylinders + displacement + horsepower + weight + 
    year + origin, data = Auto.train)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.3903 -2.2193 -0.1928  1.9964 12.9071 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.920e+01  5.617e+00  -3.419 0.000744 ***
cylinders    -3.219e-01  4.333e-01  -0.743 0.458352    
displacement  1.202e-02  1.051e-02   1.144 0.253808    
horsepower   -3.524e-02  1.533e-02  -2.299 0.022399 *  
weight       -5.300e-03  7.496e-04  -7.070 1.86e-11 ***
year          7.824e-01  6.854e-02  11.415  < 2e-16 ***
origin        1.343e+00  3.746e-01   3.586 0.000411 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.465 on 229 degrees of freedom
Multiple R-squared:  0.8087,	Adjusted R-squared:  0.8037 
F-statistic: 161.4 on 6 and 229 DF,  p-value: < 2.2e-16


In [15]:
mpg.hat.02<-predict(Auto.lm.02,newdata=Auto.test[,-1],type="response")
mpg.true<-Auto.test[,1]
HO.CV.RMS.02<-mean((mpg.true-mpg.hat.02)^2)
round(HO.CV.RMS.02,2)

[1] 10.11

In [16]:
Auto.lm.03<-lm(mpg~horsepower + weight + year + origin,data=Auto.train)
summary(Auto.lm.03)


Call:
lm(formula = mpg ~ horsepower + weight + year + origin, data = Auto.train)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.4651 -2.2980 -0.1402  1.9001 12.9841 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.976e+01  5.575e+00  -3.544 0.000477 ***
horsepower  -2.658e-02  1.257e-02  -2.114 0.035608 *  
weight      -4.927e-03  5.837e-04  -8.441 3.49e-15 ***
year         7.739e-01  6.804e-02  11.375  < 2e-16 ***
origin       1.194e+00  3.416e-01   3.495 0.000568 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.46 on 231 degrees of freedom
Multiple R-squared:  0.8076,	Adjusted R-squared:  0.8043 
F-statistic: 242.4 on 4 and 231 DF,  p-value: < 2.2e-16


In [17]:
mpg.hat.03<-predict(Auto.lm.03,newdata=Auto.test[,-1],type="response")
mpg.true<-Auto.test[,1]
HO.CV.RMS.03<-mean((mpg.true-mpg.hat.03)^2)
round(HO.CV.RMS.03,3)

[1] 10.395

In [18]:
Auto.lm.03a<-lm(mpg~ displacement + weight + year + origin,data=Auto.train)
summary(Auto.lm.03a)


Call:
lm(formula = mpg ~ displacement + weight + year + origin, data = Auto.train)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.7721 -2.2036 -0.0962  1.8303 13.1256 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -2.247e+01  5.482e+00  -4.100 5.73e-05 ***
displacement -2.983e-03  6.391e-03  -0.467  0.64117    
weight       -5.615e-03  7.431e-04  -7.556 9.65e-13 ***
year          8.102e-01  6.742e-02  12.018  < 2e-16 ***
origin        1.078e+00  3.599e-01   2.997  0.00302 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.492 on 231 degrees of freedom
Multiple R-squared:  0.8041,	Adjusted R-squared:  0.8007 
F-statistic:   237 on 4 and 231 DF,  p-value: < 2.2e-16


In [19]:
mpg.hat.03a<-predict(Auto.lm.03a,newdata=Auto.test[,-1],type="response")
mpg.true<-Auto.test[,1]
HO.CV.RMS.03a<-mean((mpg.true-mpg.hat.03a)^2)
round(HO.CV.RMS.03a,3)

[1] 10.072

### Ba.2 $k$-fold crossvalidation

In [20]:
kfoldIndexes<-function(n,k){
    l<-floor(n/k)
    Indexes<-c(1,(1:k)*l)
    Indexes[k+1]<-n
    return(Indexes)
    }

In [21]:
# Decide a 'K' for K-fold cross-validation (uppercase to avoid notation clash with the k in k-NN)
n<-nrow(Auto)
K<-5
J<-kfoldIndexes(n,K)
J
Lower<-J[-(K+1)]
Upper<-J[-1]
Lower
Upper
# Una random permutation of indexes
I<-sample(1:n)

[1]   1  78 156 234 312 392

[1]   1  78 156 234 312

[1]  78 156 234 312 392

In [22]:
# Repeat with several 'k' and choose an optimal value.
MSE.01<-rep(0,K)
for (fold in 1:K){
    Itest<-I[Lower[fold]:Upper[fold]]
    Auto.test.fold<-Auto[Itest,]
    Auto.train.fold<-Auto[-Itest,]
    Auto.lm.01.fold<-lm(mpg~.,data=Auto.train.fold)
    y.hat.01.fold<-predict(Auto.lm.01.fold,newdata=Auto.test.fold[,-1],
                           type="response")
    y.true.01.fold<-Auto.test.fold[,1]
    MSE.01.fold<-mean((y.true.01.fold-y.hat.01.fold)^2)
    MSE.01[fold]<-MSE.01.fold
    }
round(MSE.01,3)
mean.MSE.01<-mean(MSE.01)
round(mean.MSE.01,3)

[1] 14.249 11.335 12.231  9.026  9.709

[1] 11.31

### Ba.3 Leave-one-out

### Ba.4 Bootstrap

## C. `SAheart` dataset

From the `ElemStatLearn` package, `SAheart` is a data frame with 462 observations on the following 10 variables.

01. `sbp`: systolic blood pressure.

02. `tobacco`: cumulative tobacco (kg).

03. `ldl`: low density lipoprotein cholesterol.

04. `adiposity`: a numeric vector.

05. `famhist`: family history of heart disease, a factor with levels `Absent`, `Present`.

06. `typea`: type-A behavior.

07. `obesity`: a numeric vector.

08. `alcohol`: current alcohol consumption.

09. `age`: age at onset

10. `chd`: response, coronary heart disease

##### Details

A retrospective sample of males in a heart-disease high-risk region of the Western Cape, South Africa. There are roughly two controls per case of CHD. Many of the CHD positive men have undergone blood pressure reduction treatment and other programs to reduce their risk factors after their CHD event. In some cases the measurements were made after these treatments. These data are taken from a larger dataset, described in Rousseauw et al, 1983, South African Medical Journal.

In [ ]:
#install.packages("ElemStatLearn",dependencies=TRUE,repos="https://cloud.r-project.org")
require(ElemStatLearn)
data(SAheart)

In [ ]:
n<-nrow(SAheart)
ntrain<-ceiling(0.60*n)
Itrain<-sample(1:n,ntrain,replace=FALSE)
n<-nrow(SAheart)
ntrain<-ceiling(0.60*n)
Itrain<-sample(1:n,ntrain,replace=FALSE)
SAheart.train<-SAheart[Itrain,]
SAheart.test<-SAheart[-Itrain,]

## Ca. SAheart data with logistic regression

In [ ]:
SAheart.logit1<-glm(chd~.,data=SAheart.train,family=binomial)
summary(SAheart.logit1)

Prediction of probabilities that the binary 0/1 response takes the value 1 (here `chd=1`):

In [ ]:
SAheart.pred<-predict(SAheart.logit1,newdata=SAheart.test,type="response")
str(SAheart.pred)

A _crisp_ prediction of 0 or 1 is obtained from the above by taking a cut point, e.g., $L=0.5$ (this is not a mandatory value, it may depend on the problem or on the _a priori_ probabilites) and assign to 0 or 1 according to whether the probability is smaller or larger than this threshold:

In [ ]:
SAheart.pred.crisp<-1*(SAheart.pred>=0.5)
C<-table("True"=SAheart.test$chd,"Predicted"=SAheart.pred.crisp)
C

### Ca.1 Hold-out

### Ca.2 $k$-fold crossvalidation

### Ca.3 Leave-one-out

### Ca.4 Bootstrap

## Cb. `SAheart` with Fisher's linear discriminant analysis

In [ ]:
require(MASS)
SAheart.lda1<-lda(chd~.,data=SAheart.train)
SAheart.pred<-predict(SAheart.lda1,newdata=SAheart.test)
C<-table("True"=SAheart.test$chd,"Predicted"=SAheart.pred$class)
C

### Cb.1 Hold-out

### Cb.2 $k$-fold crossvalidation

### Cb.3 Leave-one-out

### Cb.4 Bootstrap

## Cb. `SAheart` with Quadratic discriminant

In [ ]:
SAheart.qda1<-qda(chd~.,data=SAheart.train)
SAheart.pred<-predict(SAheart.qda1,newdata=SAheart.test)
C<-table("True"=SAheart.test$chd,"Predicted"=SAheart.pred$class)
C

# D. `Default` data set

From the `ISLR` package. A simulated data set containing information on ten thousand customers. The aim here is to predict which customers will default on their credit card debt.

A data frame with 10000 observations on the following 4 variables.

01. `default`: A factor with levels `No` and `Yes` indicating whether the customer defaulted on their debt

02. `student`: A factor with levels `No` and `Yes` indicating whether the customer is a student

03. `balance`: The average balance that the customer has remaining on their credit card after making their monthly payment

04. `income`: Income of customer

In [ ]:
#install.packages("ISLR",dependencies=TRUE,repos="https://cloud.r-project.org")
require(ISLR)
data(Default)
str(Default)

## Da. `Default` data set with Fisher's linear discriminant

In [ ]:
n<-nrow(Default)
ntrain<-ceiling(0.6*n)
set.seed(24025)         # An arbitrary value, fixed for the sake of reproducibility of results
Itrain<-sample(1:n,ntrain,replace=FALSE)
Default.train<-Default[Itrain,]
Default.test<-Default[-Itrain,]

In [ ]:
require(MASS)
Default.lda<-lda(default~.,data=Default.train)
Default.pred<-predict(Default.lda,newdata=Default.test)
C<-table("True"=Default.test$default,"Predicted"=Default.pred$class)
C

## `Smarket` data set

This is an S&P Stock Market Data set. Daily percentage returns for the S&P 500 stock index between 2001 and 2005.
Contained in the `ISLR` package as a `data.frame` with 1250 observations on the following 9 variables.

01. `Year`: The year that the observation was recorded

02. `Lag1`: Percentage return for previous day

03. `Lag2`: Percentage return for 2 days previous

04. `Lag3`: Percentage return for 3 days previous

05. `Lag4`: Percentage return for 4 days previous

06. `Lag5`: Percentage return for 5 days previous

07. `Volume`: Volume of shares traded (number of daily shares traded in billions)

08. `Today`: Percentage return for today

09. `Direction`: A factor with levels `Down` and `Up` indicating whether the market had a positive or negative return on a given day.

In [ ]:
require(ISLR)
data(Smarket)
str(Smarket)

# Appendix: dedicated R packages

Many R packages contain functions implementing methods for validating prediction procedures, for instance:
- `caret`, 
- `CVST`, 
- `cvTools`, 
- `dprep`, 
- `sortinghat`. 

One of them (`CVST`) is devoted to a validation technique more sophisticated than the four in this notebook, and the rest are generic, with application to some prediction methods than those in our course. 